In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import spacy
from wordcloud import WordCloud
import seaborn as sns

In [ ]:
news247_turkel.shape

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
urls = []
base_url = "https://www.news247.gr/tag/ekloges-stin-toyrkia/page/"

for page_num in range(1,2):
    url = base_url + str(page_num)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('article')

    for article in articles:
        article_url = article.find('a')['href']
        if "erntogan" or "ntemirtas" or "tourkia" or "tourkoi" or "kilitsntaroglou" in article_url and "2024" not in article_url and "kaiadas" not in article_url and "meloni" not in article_url and "politiki" not in article_url and "ellada" not in article_url:
          urls.append(article_url)


print(urls)

In [ ]:
len(urls)

In [ ]:
def scrape_page (page_num, limit=24):
    base_url = f"https://www.news247.gr/tag/ekloges-stin-toyrkia/page/1"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('article')

    urls = []
    count = 0
    for article in articles:
        if count >= limit:
            break
        article_url = article.find('a')['href']
        urls.append(article_url)
        count += 1

    return urls

page_1_urls = scrape_page(1, limit=24)
print(page_1_urls)




In [ ]:
page_1_urls

In [ ]:
def scrape_page (page_num, limit=24):
    base_url = f"https://www.news247.gr/tag/ekloges-stin-toyrkia/page/2"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('article')

    urls = []
    count = 0
    for article in articles:
        if count >= limit:
            break
        article_url = article.find('a')['href']
        urls.append(article_url)
        count += 1

    return urls

# Scrape page 1 with a limit of 24 articles
page_2_urls = scrape_page(2, limit=24)
print(page_2_urls)


In [ ]:
page_2_urls

In [ ]:
def scrape_page(page_num, limit=24):
    base_url = f"https://www.news247.gr/tag/ekloges-stin-toyrkia/page/{page_num}"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('article')

    urls = []
    count = 0
    for article in articles:
        if count >= limit:
            break
        article_url = article.find('a')['href']
        urls.append(article_url)
        count += 1

    return urls


all_urls = []
for page_num in range(1, 9):
    page_urls = scrape_page(page_num, limit=24)
    all_urls.extend(page_urls)

print(all_urls)


In [ ]:
len(all_urls)


In [ ]:
news247_turkel = pd.DataFrame(all_urls)

In [ ]:
news247_turkel = news247_turkel.rename(columns={0: 'URLs'})

In [ ]:
news247_turkel

In [ ]:
full_articles_list = []
titles = []

for index, article_url in enumerate(news247_turkel['url']):
  try:

        response = requests.get(article_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        full_article_dict = {}
        full_article_dict['site'] = "news247.gr"
        full_article_dict['url'] = article_url


        title_tag = soup.find('h1', {"class": "single_article__title article__small_title article__title"})
        title = title_tag.text.strip() if title_tag else 'No Title Found'
        full_article_dict['title'] = title


        category_tag = soup.find("a", {"title": True})
        category = category_tag['title'].strip() if category_tag else 'No Category Found'
        full_article_dict['category'] = category

        author_tag = soup.find("span", {"class": "post__author"})
        author = author_tag.text.strip() if author_tag else 'No Author Found'
        full_article_dict['author'] = author

        datetime_tag = soup.find("li", {"class": "article__date s-font"})
        datetime = datetime_tag.text.strip() if datetime_tag else 'No Date/Time Found'
        full_article_dict['datetime'] = datetime

        summary_tag = soup.find("h2", {"class": "article__lead"})
        summary = summary_tag.text.strip() if summary_tag else 'No Text Found'
        full_article_dict["summary"] = summary


        text_tag = soup.find("div", {"class": "paragraph"})
        text = text_tag.text.strip() if text_tag else 'No Text Found'
        full_article_dict['text'] = text


        full_articles_list.append(full_article_dict)


        print(f"Processed {index + 1}/{len(news247_turkel)}: {article_url}")


        time.sleep(0.200)

  except Exception as e:
        print(f"Error processing {article_url}: {e}")
        full_article_dict['title'] = 'Error'
        full_article_dict['category'] = 'Error'
        full_article_dict['author'] = 'Error'
        full_article_dict['datetime'] = 'Error'
        full_article_dict['text'] = 'Error'
        full_articles_list.append(full_article_dict)

news247_turkel = pd.DataFrame(full_articles_list)

In [ ]:
pd.set_option('display.max_rows', 192)

In [ ]:
news247_turkel

In [ ]:
#Διορθώνοντας τα NaN values
news247_turkel[news247_turkel['title'] == 'No Title Found']

In [ ]:
news247_turkel["text"][43] = "Ο Ρετζέπ Ταγίπ Ερντογάν είναι ο κυρίαρχος του παιχνιδιού στην Τουρκία, κόντρα στις δημοσκοπήσεις και την ετερόκλητη αντιπολίτευση, που για πρώτη φορά συνασπίστηκε εναντίον του, στο πρόσωπο του Κεμάλ Κιλιτσντάρογλου. Το NEWS247 πέρασε μια εβδομάδα στην Κωνσταντινούπολη, επιχειρώντας να καταλάβει πώς σκέφτονται οι άνθρωποι πριν την κρισιμότερη εκλογική αναμέτρηση τελευταίων δεκαετιών. Πώς επηρέασε την ψήφο ο πρόσφατος καταστροφικός σεισμός; Γιατί η τιμή του κρεμμυδιού εξελίχθηκε σε σύμβολο της κάλπης; Γιατί, όποιο κι αν είναι το αποτέλεσμα, η Τουρκία του Ερντογάν θα είναι αύριο μια άλλη χώρα;"

In [ ]:
news247_turkel.to_csv("/content/drive/MyDrive/news247_turkel", index = False)

In [ ]:
news247_turkel = pd.read_csv("/content/drive/MyDrive/news247_turkel")

In [ ]:
news247_turkel

In [ ]:
news247_turkel["text"] = news247_turkel["text"].str.replace("\n", "")
news247_turkel['text'] = news247_turkel["text"] .str.replace("\n", "")
news247_turkel["text"]  = news247_turkel['text'].str.replace("\nΑκολουθήστε το News 24/7 στο Google News και μάθετε πρώτοι όλες τις ειδήσεις'","")
news247_turkel["text"]  = news247_turkel["text"].str.replace("Ακολουθήστε το News 24/7 στο Google News και μάθετε πρώτοι όλες τις ειδήσεις", "")
news247_turkel["text"] = news247_turkel["text"].str.replace("pic.twitter.com/TjxKUU8QgJ\n—", "")
news247_turkel["text"] = news247_turkel["text"].str.replace("🔴Siirt Pervari’de “ikinci bir duyuruya kadar” sokağa çıkma yasağı ilan edildi! pic.twitter.com/QYVmJqDxdZ— Haber 7 (@Haber7)","")
news247_turkel["text"] = news247_turkel["text"].str.replace("\xa0", "")
news247_turkel["text"] = news247_turkel["text"].str.replace(  "AP Photo        \r", "")
news247_turkel["text"] = news247_turkel["text"].str.replace("\r", "")
news247_turkel["text"] = news247_turkel["text"].str.replace("pic.twitter.com/Z2A9gnfTq6— Ekrem İmamoğlu (International) (@imamoglu_int) March 31, 2024","")
news247_turkel["text"] = news247_turkel["text"].str.replace("Tonight, #Istanbul has spoken clearly: our collective desire for #democracy shines brighter than ever, marking a pivotal step towards #unity and the end of polarization.At a time when the world witnesses a retreat from democratic values, our city stands as a beacon of hope and…","")
news247_turkel["text"] = news247_turkel["text"].str.replace("Ακολουθήστε το News 24/7 στοGoogle Newsκαι μάθετε πρώτοι όλες τιςειδήσεις","")
news247_turkel["text"] = news247_turkel["text"].str.replace("Pollitics – Όλες οι δημοσκοπήσεις σε μία σελίδα με αναλυτικά γραφήματαΑποτελέσματα Εκλογών 2023: Αναλυτικά τα αποτελέσματα με χάρτη επικράτειας και περιφερειών για τις εθνικές εκλογές 2023","")
news247_turkel["text"] = news247_turkel["text"].str.replace("Ανταπόκριση του NEWS 24/7 από την Κωνσταντινούπολη","")



In [ ]:
news247_turkel["text"][26]

In [ ]:
def replace_month_name(date_str):
    months = {
        "Ιανουαρίου": "01",
        "Φεβρουαρίου": "02",
        "Μαρτίου": "03",
        "Απριλίου": "04",
        "Μαΐου": "05",
        "Ιουνίου": "06",
        "Ιουλίου": "07",
        "Αυγούστου": "08",
        "Σεπτεμβρίου": "09",
        "Οκτωβρίου": "10",
        "Νοεμβρίου": "11",
        "Δεκεμβρίου": "12"
    }
    for greek_month, month_num in months.items():
        if greek_month in date_str:
            return date_str.replace(greek_month, month_num)
    return date_str

news247_turkel['datetime'] = news247_turkel['datetime'].apply(replace_month_name)
news247_turkel['datetime'] = pd.to_datetime(news247_turkel['datetime'], format='%d %m %Y %H:%M', errors='coerce') #για την περίπτωση που έχω missing values

print(news247_turkel)


In [ ]:
news247_turkel

In [ ]:
news247_turkel.to_csv("/content/drive/MyDrive/news247_turkel", index = False)

In [ ]:
news247_turkel = pd.read_csv("/content/drive/MyDrive/news247_turkel")

In [ ]:
news247_turkel;

In [ ]:
news247_turkel['year'] = pd.DatetimeIndex(news247_turkel['datetime']).year
news247_turkel['month'] = pd.DatetimeIndex(news247_turkel['datetime']).month
news247_turkel['day'] = pd.DatetimeIndex(news247_turkel['datetime']).day
news247_turkel['hour'] = pd.DatetimeIndex(news247_turkel['datetime']).hour

In [ ]:
news247_turkel

In [ ]:
news247_turkel['hour'].value_counts().plot(kind='bar')

In [ ]:
news247_turkel['day'].value_counts().plot(kind='bar')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import spacy
from wordcloud import WordCloud
import seaborn as sns

In [ ]:
!python -m spacy download el_core_news_sm

In [ ]:
news247_turkel_full_text = news247_turkel['text'].str.cat(sep = ' ')
news247_turkel_full_text

In [ ]:
nlp = spacy.load('el_core_news_sm')


In [ ]:
wordcloud1 = WordCloud(
    stopwords = nlp.Defaults.stop_words,
    width = 2000,
    height = 1000,
    background_color = 'black'
 ).generate(news247_turkel_full_text)
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud1, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
full_doc = nlp(news247_turkel_full_text)

In [ ]:
full_doc[:200]

In [ ]:
lemmatized_text = ' '.join(token.lemma_ for token in full_doc)

In [ ]:
wordcloud2 = WordCloud(
    stopwords = nlp.Defaults.stop_words,
    width = 2000,
    height = 1000,
    background_color = 'black'
 ).generate(lemmatized_text)
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud2, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
news247_turkel = pd.read_csv('news247_turkish_elections')

In [ ]:
news247_turkel.to_csv('news247_turkish_elections', index = False)

In [ ]:
news247_turkel

In [ ]:
pd.set_option('display.max_rows', 193)

In [ ]:
short_df = news247_turkel[["title", "datetime"]]

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
short_df

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
nlp = spacy.load('el_core_news_md')

In [ ]:
news247_turkel_full_text = news247_turkel['text'].str.cat(sep = ' ')

In [ ]:
news247_turkel_full_doc = nlp(news247_turkel_full_text)

In [ ]:
lemmatized_text = ' '.join(token.lemma_ for token in news247_turkel_full_doc)

In [ ]:
stopwords = nlp.Defaults.stop_words
stopwords.add("ς")
stopwords.add("μπορώ")
stopwords.add("αναφέρω")
stopwords.add("υπάρχω")
stopwords.add("γίνομαι")
stopwords.add("ή")
stopwords.add("κάνω")
stopwords.add("θέλω")
stopwords.add("κ")
stopwords.add("λέγω")

In [ ]:
lemmatized_text = lemmatized_text.replace("Κεμάλς Κιλιτσντάρογλου", "Κεμάλ Κιλιτσντάρογλου")
wordcloud_news247_turkel = WordCloud(
    stopwords=nlp.Defaults.stop_words,
    width=2000,
    height=1000,
    background_color='black'
).generate(lemmatized_text)

fig = plt.figure(
    figsize=(40, 30),
    facecolor='k',
    edgecolor='k'
)
plt.imshow(wordcloud_news247_turkel, interpolation='bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), min_df=0.01, max_df=0.95)

In [ ]:
news247_turkel_clean = news247_turkel.dropna(subset=["text"])
count_vector = cv.fit_transform(news247_turkel_clean["text"])


In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=100, ngram_range=(2,5))
count_vector = cv.fit_transform(news247_turkel_clean["text"])
news247_turkel_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out()) 

In [ ]:
news247_turkel_bigrams.sum(axis =0).sort_values(ascending = False)

In [ ]:
data = {"Ταγίπ Ερντογάν" :                                     169,
"Κεμάλ Κιλιτσντάρογλου" :                                     158,
"Ρετζέπ Ταγίπ Ερντογάν" :                                     140,
"προεδρικές εκλογές/προεδρικών εκλογών"  :                     103,
"Τούρκος Πρόεδρος/Τούρκο Πρόεδρο"  :                           73,
"δεύτερο γύρο"     :                                           59,
"τουρκικές εκλογές" :                                          57,
"14ης μαΐου" :                                                 47,
"news247 καλύπτει τουρκικές εκλογές"  :                        37,
"τουρκικές εκλογές κωνσταντινούπολη διαβάστε ανταπόκριση":     35,
"βουλευτικές εκλογές" :                                        30,
"ρεπουμπλικανικού λαϊκού κόμματος"  :                          30,
"Εκρέμ Ιμάμογλου" :                                            29,
"Σινάν Ογάν"  :                                                28,
"ανατολική μεσόγειο" :                                         27,
"εκλογές 14ης μαΐου" :                                         26,
"28 μαΐου" :                                                   26,
"δικαιοσύνης ανάπτυξης" :                                      24,
"Μεράλ Ακσενέρ"  :                                             23,
"κομμάτων αντιπολίτευσης" :                                    23,
"20 χρόνια"  :                                                 23,
"εξωτερική πολιτική"  :                                        23,
"εκλογών τουρκία"   :                                          22,
"προεκλογική εκστρατεία" :                                     22,
"14 μαΐου"  :                                                  21,
"γύρο προεδρικών εκλογών"  :                                   21,                                         }

df = pd.DataFrame(list(data.items()), columns=['Διγράμματα / Πολυγράμματα', 'Αναφορές'])

df = df.sort_values(by='Αναφορές', ascending=False)

plt.figure(figsize=(10, 8))

bars = plt.barh(df['Διγράμματα / Πολυγράμματα'], df['Αναφορές'], color='crimson', edgecolor= 'black')
for bar in bars:
    plt.text(
        bar.get_width() + 2, bar.get_y() + bar.get_height() / 2,  
        str(bar.get_width()),
        va='center',
        ha='left',  
        color='black',
        fontsize=10
    )
    
plt.xlabel('Αναφορές')
plt.ylabel('Διγράμματα / Πολυγράμματα')
plt.xlim(0,200)
plt.title('Διγράμματα/Πολυγράμματα Τουρκικών Εκλογών 2023 - News247')
plt.gca().invert_yaxis() 
plt.show()

In [ ]:
filepath = "https://raw.githubusercontent.com/datajour-gr/DataJournalism/main/Bachelor%20Lessons%202023/Lesson%2010/NRC_GREEK_Translated_6_2020.csv"

In [ ]:
emolex_df = pd.read_csv(filepath)

In [ ]:
emolex_df = emolex_df.drop_duplicates(subset=['word'])
emolex_df = emolex_df.dropna()
emolex_df.reset_index(inplace = True, drop=True)

In [ ]:
vec = CountVectorizer(analyzer = 'word', vocabulary = emolex_df.word,
                      lowercase=False, 
                      strip_accents = 'unicode',  
                      stop_words= list(nlp.Defaults.stop_words),
                      ngram_range=(1, 2))

In [ ]:
matrix = vec.fit_transform(news247_turkel_clean["text"])
vocab = vec.get_feature_names_out()
wordcount_df = pd.DataFrame(matrix.toarray(), columns=vocab)

In [ ]:
positive_words = emolex_df[emolex_df.Positive == 1]['word']

In [ ]:
negative_words = emolex_df[emolex_df.Negative == 1]['word']

In [ ]:
news247_turkel['positive_text'] = wordcount_df[positive_words].sum(axis=1)

In [ ]:
news247_turkel['negative_text'] = wordcount_df[negative_words].sum(axis=1)

In [ ]:
news247_turkel['pos/neg_text'] = news247_turkel['positive_text'] - news247_turkel['negative_text']

In [ ]:
news247_turkel['pos/neg_text'].mean()

In [ ]:
joy_words = emolex_df[emolex_df.Joy == 1]['word']

In [ ]:
anger_words = emolex_df[emolex_df.Anger == 1]['word']

In [ ]:
news247_turkel['joy_text'] = wordcount_df[joy_words].sum(axis=1)

In [ ]:
news247_turkel['anger_text'] =  wordcount_df[anger_words].sum(axis=1)

In [ ]:
news247_turkel['joy/anger_text'] = news247_turkel['joy_text'] - news247_turkel['anger_text']

In [ ]:
news247_turkel['joy/anger_text'].mean()

In [ ]:
fear_words = emolex_df[emolex_df.Fear == 1]['word']

In [ ]:
trust_words = emolex_df[emolex_df.Trust == 1]['word']

In [ ]:
news247_turkel['fear_text'] = wordcount_df[fear_words].sum(axis=1)


In [ ]:
news247_turkel['trust_text'] = wordcount_df[trust_words].sum(axis=1)

In [ ]:
news247_turkel['trust/fear_text'] = news247_turkel['trust_text'] - news247_turkel['fear_text']

In [ ]:
news247_turkel['trust/fear_text'].mean()

In [ ]:
matrix = vec.fit_transform(news247_turkel["title"])
vocab = vec.get_feature_names_out()
wordcount_df = pd.DataFrame(matrix.toarray(), columns=vocab)

In [ ]:
news247_turkel['positive_title'] = wordcount_df[positive_words].sum(axis=1)

In [ ]:
news247_turkel['negative_title'] = wordcount_df[negative_words].sum(axis=1)

In [ ]:
news247_turkel['pos/neg_title'] = news247_turkel['positive_title'] - news247_turkel['negative_title']

In [ ]:
news247_turkel['pos/neg_title'].mean()

In [ ]:
news247_turkel['joy_title'] = wordcount_df[joy_words].sum(axis=1)

In [ ]:
news247_turkel['anger_title'] =  wordcount_df[anger_words].sum(axis=1)

In [ ]:
news247_turkel['joy/anger_title'] = news247_turkel['joy_title'] - news247_turkel['anger_title']

In [ ]:
news247_turkel['joy/anger_title'].mean()

In [ ]:
news247_turkel['fear_title'] = wordcount_df[fear_words].sum(axis=1)

In [ ]:
news247_turkel['trust_title'] = wordcount_df[trust_words].sum(axis=1)

In [ ]:
news247_turkel['trust/fear_title'] = news247_turkel['trust_title'] - news247_turkel['fear_title']

In [ ]:
news247_turkel['trust/fear_title'].mean()

In [ ]:
means = {
    "Θετικότητα/Αρνητικότητα": {
        'Πολικότητα Κειμένων': news247_turkel['pos/neg_text'].mean().round(2),
        'Πολικότητα Τίτλων': news247_turkel['pos/neg_title'].mean().round(2)
    },
    'Εμπιστοσύνη/Φόβος': {
        'Πολικότητα Κειμένων': news247_turkel['trust/fear_text'].mean().round(2),
        'Πολικότητα Τίτλων': news247_turkel['trust/fear_title'].mean().round(2)
    },
    'Χαρά/Θυμός': {
        'Πολικότητα Κειμένων': news247_turkel['joy/anger_text'].mean().round(2),
        'Πολικότητα Τίτλων': news247_turkel['joy/anger_title'].mean().round(2),
    }
}


means_df = pd.DataFrame(means).T


color_map = {
    "Θετικότητα/Αρνητικότητα": 'green',
    'Εμπιστοσύνη/Φόβος': 'lightblue',
    'Χαρά/Θυμός': 'crimson'
}


new_col_titles = ['Πολικότητα Κειμένων', 'Πολικότητα Τίτλων']


fig, ax = plt.subplots(figsize=(14, 8))


ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.set_frame_on(False)


table = ax.table(
    cellText=means_df.values,
    rowLabels=means_df.index,
    colLabels=new_col_titles,
    cellLoc='center',
    loc='center',
    colColours=['lightgrey'] * len(new_col_titles),
    rowColours=['white'] * len(means_df.index)
)


for i, label in enumerate(means_df.index):
    row_color = color_map.get(label, 'white') 
    for j in range(len(new_col_titles)):
        table[(i + 1, j)].set_facecolor(row_color)


table.auto_set_font_size(False)
table.set_fontsize(14)
table.scale(1.5, 3)


plt.show()

In [ ]:
titles_erdogan = news247_turkel[news247_turkel['title'].str.contains('Ερντογάν', case=False, na=False)]

In [ ]:
titles_erdogan.shape

In [ ]:
titles_Κιλιτσντάρογλου= news247_turkel[news247_turkel['title'].str.contains('Κιλιτσντάρογλου', case=False, na=False)]

In [ ]:
titles_Κιλιτσντάρογλου.shape

In [ ]:
titles_with_both = news247_turkel[
    news247_turkel['title'].str.contains(r'Ερντογάν.*Κιλιτσντάρογλου|Κιλιτσντάρογλου.*Ερντογάν', case=False, na=False)
]

titles_with_both.shape


In [ ]:
news247_turkel.shape

In [ ]:
pip install matplotlib-venn


In [ ]:
from matplotlib_venn import venn2
count_erdogan_only = 96 - 16
count_kilicdaroglu_only = 32 - 16
count_both = 16
total_titles = 172


count_neither = total_titles - (count_erdogan_only + count_kilicdaroglu_only + count_both)


plt.figure(figsize=(10, 8))

venn = venn2(subsets=(count_erdogan_only, 
                      count_kilicdaroglu_only, 
                      count_both), 
             set_labels=('Ερντογάν', 'Κιλιτσντάρογλου'))


plt.title('Δημοσιεύματα για Ερντογάν-Κιλιτσντάρογλου -News247')

plt.text(0.5, -0.3, f'Σύνολο Άρθρων: {total_titles}', ha='center', va='center', fontsize=8, color='black')


plt.show()


In [ ]:
count_vector = cv.fit_transform(titles_erdogan["title"])

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=100, ngram_range=(2,2))
count_vector = cv.fit_transform(titles_erdogan["title"])
titles_erdogan_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out())

In [ ]:
titles_erdogan_bigrams.sum(axis =0).sort_values(ascending = False)

In [ ]:
count_vector = cv.fit_transform(titles_Κιλιτσντάρογλου["title"])

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=100, ngram_range=(2,2))
count_vector = cv.fit_transform(titles_Κιλιτσντάρογλου["title"])
titles_Κιλιτσντάρογλου_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out())

In [ ]:
titles_Κιλιτσντάρογλου_bigrams.sum(axis =0).sort_values(ascending = False)

In [ ]:
import geopandas as gpd


shapefile_path = r"C:\Users\stath\OneDrive\Υπολογιστής\ne_110m_admin_0_countries\ne_110m_admin_0_countries.shp"


world = gpd.read_file(shapefile_path)


greece_turkey = world[world["SOVEREIGNT"].isin(['Greece', 'Turkey'])]


fig, ax = plt.subplots(figsize=(20, 10))  # Increased size


greece_turkey[greece_turkey["SOVEREIGNT"] == "Greece"].plot(ax=ax, color='blue', edgecolor='black', label='Greece')
greece_turkey[greece_turkey["SOVEREIGNT"] == "Turkey"].plot(ax=ax, color='red', edgecolor='black', label='Turkey')


ax.set_title('Τίτλοι δημοσιευμάτων για Μητσοτάκη(ΝΔ)-Τσίπρα(ΣΥΡΙΖΑ), Ερντογάν-Κιλιτσντάρογλου - News247', fontsize=17)


ax.grid(True, linestyle='--', alpha=0.7)


greece_coords = (19.0, 36.5) 
turkey_coords = (33.0, 37.5)  

# Phrases for Greece
greece_phrases = [
      
     "Τσίπρας: (αυτούσια δήλωση)",
    
     'ΝΔ ψάχνουν “πιο σοβαρή Χρυσή Αυγή”',
      '"δόγμα Μητσοτάκη"',
      "άκομψη υπόδειξη Γεραπετρίτη σε δημοσιογράφο",
     "εκβιασμός εργοδότη σε υπαλλήλους για να ψηφίσουν ΝΔ"
]


turkey_phrases = [
     "ακατάδεκτος Ερντογάν",
     "ακροδεξιό παραλήρημα Κιλιτσντάρογλου",
     "οικονομική αναταραχή",
     "ανησυχία δυτικών ΜΜΕ",
     "δύο Τουρκίες"
]


for i, phrase in enumerate(greece_phrases):
    ax.text(greece_coords[0], greece_coords[1] + i*0.7, phrase, fontsize=12, color='black', ha='left', weight='bold')


for i, phrase in enumerate(turkey_phrases):
    ax.text(turkey_coords[0], turkey_coords[1] + i*0.7, phrase, fontsize=14, color='black', ha='center', weight='bold')


ax.legend()

plt.show()

In [ ]:
import re

frames = {
    'Σύγκρουση': [
        'κόντρα', 'VS', 'μάχη', 'επίθεση', 'αντιπαράθεση', '«σφάχτηκαν»', '«επιτίθεται»', "ή", "δίλημμα",
        '«πυρά»', '«Πυρά»','πυρά', 'Πυρά', 'μπηχτή', 'μεταξύ', 'σύγκρουση', '«σκοτωμός»', 'απαντά', '«σπόντα»',
        '«μάχης»', 'μια πλευρά', 'Μητσοτάκη-Τσίπρα', 'απάντηση', 'καυγά', 'αίμα', 'απαντάμε',
        'διαφορά', 'έναντι', 'σύγκριση', 'κατά', 'εναντίον','προβάδισμα', 'Μητσοτάκης-Τσίπρας', 'ΝΔ-ΣΥΡΙΖΑ' 'ΝΔ - ΣΥΡΙΖΑ', "Ερντογάν - Κιλιτσντάρογλου, Ερντογάν-Κιλιτσντάρογλου", "ελληνοτουρκικά"
    ]
}

def detect_conflict_frames(title, frames):
    conflict_keywords = frames['Σύγκρουση']
    
    pattern = r'\b(?:' + '|'.join(map(re.escape, conflict_keywords)) + r')\b'

    return bool(re.search(pattern, title))


news247_turkel['Conflict'] = news247_turkel['title'].apply(lambda x: 1 if detect_conflict_frames(x, frames) else 0)

In [ ]:
news247_turkel['Conflict'].value_counts()

In [ ]:
news247_turkel.shape

In [ ]:
7.55

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt


data = {
    'Ιστοσελίδα': ['Iefimerida', 'NewsIT', 'News247', 'PressProject'],
    'Ελληνικές εκλογές %': [11.62, 11.07, 6.58, 6.96],
    'Τουρκικές εκλογές %': [12.72, 8.90, 7.55, "-"]  
}


df = pd.DataFrame(data)


colors = {
    'Iefimerida': 'black',
    'NewsIT': 'darkorange',
    'News247': 'red',
    'PressProject': 'lightblue'
}


fig, ax = plt.subplots(figsize=(12, 4))  # Adjust figure size


ax.axis('off')
ax.axis('tight')


table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')


table.auto_set_font_size(False)
table.set_fontsize(12)


table.scale(1.2, 1.2)


for i, website in enumerate(df['Ιστοσελίδα']):
    row_color = colors.get(website, 'white')  
    for j in range(len(df.columns)):
        cell = table[(i + 1, j)]
        cell.set_facecolor(row_color)
      
        if website == 'Iefimerida':
            cell.set_text_props(color='white')


for key, cell in table.get_celld().items():
    if key[0] == 0:
        cell.set_text_props(weight='bold', color='white')
        cell.set_facecolor('grey')


plt.title('Χρήση Πλαισίου Σύγκρουσης στις Ελληνικές και Τουρκικές Εκλογές ανά Ιστοσελίδα', fontsize=14)
plt.show()

